In [ ]:
import cv2, numpy as np, matplotlib.pyplot as plt
from IPython.display import clear_output

CAMERA_INDEX = 0
FRAME_WIDTH, FRAME_HEIGHT = 1280, 720

def _gamma(img, g=1.2):
    lut = np.array([((i/255.0)**(1.0/g))*255 for i in range(256)]).astype(np.uint8)
    return cv2.LUT(img, lut)

haar = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def haar_detect_boxes_rgb(rgb):
    H,W = rgb.shape[:2]
    SCALE = 0.6  # 0.4〜0.8で調整可
    small = cv2.resize(rgb, (int(W*SCALE), int(H*SCALE)), interpolation=cv2.INTER_AREA)
    gray  = cv2.cvtColor(small, cv2.COLOR_RGB2GRAY)
    gray  = cv2.equalizeHist(gray)
    gray  = _gamma(gray, g=1.1)
    faces = haar.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=4, minSize=(60,60))
    boxes = []
    for (x,y,w,h) in faces:
        l = int(x / SCALE); t = int(y / SCALE)
        r = int((x+w) / SCALE); b = int((y+h) / SCALE)
        boxes.append((t,r,b,l))
    return boxes

def hog_detect_boxes_rgb(rgb, up=1):
    import face_recognition
    return face_recognition.face_locations(rgb, number_of_times_to_upsample=up, model="hog")

# 撮影してプレビュー
cap = cv2.VideoCapture(CAMERA_INDEX, cv2.CAP_AVFOUNDATION)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)

# 露出安定のため数フレーム捨てる
for _ in range(6):
    cap.read()

ok, frame = cap.read()
cap.release()
assert ok, "カメラからフレームを取得できませんでした"

rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
vis = rgb.copy()

boxes = haar_detect_boxes_rgb(rgb)
fallback = False
if not boxes:
    # HOGにフォールバック（重いが見つかりやすい）
    try:
        boxes = hog_detect_boxes_rgb(rgb, up=1)
        fallback = True
    except Exception as e:
        print("HOG fallback error:", e)

for (t,r,b,l) in boxes:
    cv2.rectangle(vis, (l,t), (r,b), (0,255,0), 2)

clear_output(wait=True)
plt.figure(figsize=(7,5))
plt.imshow(vis); plt.axis("off")
plt.title(f"検出数: {len(boxes)}  ({'HOG' if fallback else 'Haar'})")
plt.show()

print("検出ボックス:", boxes)

In [ ]:
import json, uuid
from pathlib import Path
from datetime import datetime
import cv2, numpy as np, face_recognition

PEOPLE_DIR = Path("face_data/people"); PEOPLE_DIR.mkdir(parents=True, exist_ok=True)
CAMERA_INDEX = 0
FRAME_WIDTH, FRAME_HEIGHT = 1280, 720

haar = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def _haar_boxes(rgb):
    H,W = rgb.shape[:2]
    SCALE = 0.6
    small = cv2.resize(rgb, (int(W*SCALE), int(H*SCALE)), interpolation=cv2.INTER_AREA)
    gray  = cv2.cvtColor(small, cv2.COLOR_RGB2GRAY)
    gray  = cv2.equalizeHist(gray)
    faces = haar.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=4, minSize=(60,60))
    boxes = []
    for (x,y,w,h) in faces:
        l = int(x / SCALE); t = int(y / SCALE)
        r = int((x+w) / SCALE); b = int((y+h) / SCALE)
        boxes.append((t,r,b,l))
    return boxes

def _hog_boxes(rgb, up=1):
    return face_recognition.face_locations(rgb, number_of_times_to_upsample=up, model="hog")

def detect_boxes_robust(rgb):
    boxes = _haar_boxes(rgb)
    if boxes:
        return boxes, "Haar"
    boxes = _hog_boxes(rgb, up=1)
    if boxes:
        return boxes, "HOGx1"
    boxes = _hog_boxes(rgb, up=2)
    return boxes, "HOGx2" if boxes else ([], "None")

def register_person_robust(name: str, meta: dict, shots: int = 12, interval: float = 0.2):
    cap = cv2.VideoCapture(CAMERA_INDEX, cv2.CAP_AVFOUNDATION)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)

    encs, last_crop, used = [], None, None

    # 露出安定
    for _ in range(6):
        cap.read()

    for i in range(shots):
        ok, frame = cap.read()
        if not ok: break
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        boxes, used = detect_boxes_robust(rgb)
        if boxes:
            t,r,b,l = max(boxes, key=lambda x:(x[2]-x[0])*(x[1]-x[3]))
            vecs = face_recognition.face_encodings(rgb, known_face_locations=[(t,r,b,l)], num_jitters=1)
            if vecs:
                encs.append(vecs[0].astype(np.float64))
                last_crop = frame[t:b, l:r]
        cv2.waitKey(int(interval*1000))
    cap.release()

    if not encs:
        print("⚠️ 顔が検出できませんでした。明るさ/距離/正面を調整して再試行。（検出器:", used, "）")
        return

    pid = uuid.uuid4().hex[:12]
    pdir = PEOPLE_DIR / f"{pid}_{name}"
    pdir.mkdir(parents=True, exist_ok=True)
    np.save(str(pdir/"encodings.npy"), np.vstack(encs))
    (pdir/"faces").mkdir(parents=True, exist_ok=True)
    if last_crop is not None:
        now_ts = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        cv2.imwrite(str(pdir/"faces"/f"{now_ts}.jpg"), last_crop)
    meta_doc = {"name": name, "meta": meta, "created_at": datetime.utcnow().isoformat()}
    (pdir/"meta.json").write_text(json.dumps(meta_doc, ensure_ascii=False, indent=2), encoding="utf-8")

    print(f"✅ 登録完了: {name}  shots={len(encs)}  detector={used}  dir={pdir}")


In [ ]:
register_person_robust(
    "ishioka",
    {
        "出身": "札幌",
        "学年": "M1",
        "所属": "UTokyo",
        "研究テーマ": "津波予測モデル・反射波解析",
        "話した内容": "津波のAR可視化アプリ開発について議論。Kamchatka 2025想定の波源モデル精度の話も。",
        "第一印象": "明るくて話しやすく、技術への興味が強い。",
        "覚えておきたいこと": [
            "Python と iOS の両方できる",
            "地震研所属",
            "顔認識アプリにも興味あり"
        ]
    },
    shots=12,
    interval=0.2
)


In [ ]:
# === FaceID: 5秒キャプチャ → 認識 → 画像/音声付きでMail.app自動送信（シンプルTTS） ===
# 依存: opencv-python, face_recognition, numpy, pillow, macOS の say コマンド (標準)
import os, json, subprocess
from pathlib import Path
from datetime import datetime
import numpy as np, cv2, face_recognition
from PIL import Image, ImageDraw, ImageFont

# ====== 設定 ======
RECIPIENT = "6301043919@utac.u-tokyo.ac.jp"   # ←送り先を必要に応じて変更
CAMERA_INDEX = 0                               # 内蔵=0, 外付=1 など
FRAME_WIDTH, FRAME_HEIGHT = 1280, 720
FACE_TOLERANCE = 0.55                          # 0.47〜0.55（低いほど厳密）

# ====== 日本語ラベル（Pillow） ======
def _find_jp_font():
    for p in [
        "/System/Library/Fonts/Hiragino Sans W3.ttc",
        "/System/Library/Fonts/ヒラギノ角ゴシック W3.ttc",
        "/System/Library/Fonts/Hiragino Sans W6.ttc",
        "/Library/Fonts/NotoSansCJKjp-Regular.otf",
        "/Library/Fonts/NotoSansCJK-Regular.ttc",
        "/Library/Fonts/Arial Unicode.ttf",
    ]:
        if os.path.exists(p):
            return p
    return None

_JP_FONT = _find_jp_font()

def draw_label_jp(img_rgb, text, xy, font_size=18, pad=4,
                  fg=(255, 255, 255), bg=(0, 0, 0)):
    font = ImageFont.truetype(_JP_FONT, font_size) if _JP_FONT else ImageFont.load_default()
    img_pil = Image.fromarray(img_rgb)
    draw = ImageDraw.Draw(img_pil)
    x, y = xy
    bbox = draw.textbbox((0, 0), text, font=font)
    tw, th = bbox[2] - bbox[0], bbox[3] - bbox[1]
    draw.rectangle([x - pad, y - pad, x + tw + pad, y + th + pad], fill=bg)
    draw.text((x, y), text, font=font, fill=fg)
    return np.array(img_pil)

# ====== 既知データ読み込み（ベクトル単位でフラット化） ======
def load_known_vectors():
    base = Path("face_data/people")
    enc_paths = list(base.glob("*/encodings.npy"))
    known_encs, owner_by_vec, meta_by_vec = [], [], []
    for ep in enc_paths:
        person_name = ep.parent.name.split("_", 1)[-1]
        meta_obj = {}
        mp = ep.parent / "meta.json"
        if mp.exists():
            try:
                meta_all = json.loads(mp.read_text(encoding="utf-8"))
                meta_obj = meta_all.get("meta", {})
            except Exception:
                meta_obj = {}
        arr = np.load(str(ep))
        if arr.ndim == 1 and arr.shape[0] == 128:
            arr = arr.reshape(1, 128)
        for row in arr:
            known_encs.append(row.astype(np.float64))
            owner_by_vec.append(person_name)
            meta_by_vec.append(meta_obj)
    return (np.array(known_encs) if known_encs else None), owner_by_vec, meta_by_vec

# ====== Mail.app を AppleScript で自動送信 ======
def send_via_mail_app(recipient, subject, body, attachments):
    tmp_script = Path("send_mail.scpt")
    attach_lines = "\n".join(
        [f'set end of attchs to (POSIX file "{str(Path(p).resolve())}")'
         for p in attachments]
    )
    subject_apple = subject.replace('"', "'")
    body_apple = body.replace('"', "'")
    script_content = f"""
    on run
        set theRecipient to "{recipient}"
        set theSubject to "{subject_apple}"
        set theContent to "{body_apple}"
        set attchs to {{}}
{attach_lines}
        tell application "Mail"
            set newMsg to make new outgoing message with properties {{visible:false, subject:theSubject, content:theContent & return & return}}
            tell newMsg
                make new to recipient at end of to recipients with properties {{address:theRecipient}}
                repeat with f in attchs
                    try
                        make new attachment with properties {{file name:f}} at after the last paragraph
                    end try
                end repeat
                send
            end tell
        end tell
    end run
    """
    tmp_script.write_text(script_content, encoding="utf-8")
    subprocess.run(["osascript", str(tmp_script)], check=True)
    tmp_script.unlink(missing_ok=True)

# ====== macOS TTS（AI音声） ======
def make_tts_audio(text: str) -> Path:
    Path("share_out").mkdir(exist_ok=True)
    stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_path = Path(f"share_out/voice_{stamp}.aiff")
    # 日本語音声（Kyoko など）にしたい場合は -v オプション指定可: ["say","-v","Kyoko",...]
    subprocess.run(["say", text, "-o", str(out_path)], check=True)
    return out_path

# ====== 顔検出（Haar → HOG フォールバック） ======
def haar_boxes(rgb):
    haar = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    H, W = rgb.shape[:2]
    SCALE = 0.6
    small = cv2.resize(rgb, (int(W * SCALE), int(H * SCALE)))
    gray = cv2.cvtColor(small, cv2.COLOR_RGB2GRAY)
    gray = cv2.equalizeHist(gray)
    faces = haar.detectMultiScale(gray, scaleFactor=1.05,
                                  minNeighbors=4, minSize=(60, 60))
    boxes = []
    for (x, y, w, h) in faces:
        l = int(x / SCALE)
        t = int(y / SCALE)
        r = int((x + w) / SCALE)
        b = int((y + h) / SCALE)
        boxes.append((t, r, b, l))
    return boxes

def hog_boxes(rgb, up=1):
    return face_recognition.face_locations(
        rgb, number_of_times_to_upsample=up, model="hog"
    )

# ====== メイン：5秒撮影→最も明るいフレームで認識→メール送信（シンプルTTS） ======
def identify_and_email(capture_time: float = 5.0):
    import time

    def brightness(img):
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        return gray.mean()

    # 既知データ
    known_encs, owners, metas = load_known_vectors()

    # カメラ起動
    cap = cv2.VideoCapture(CAMERA_INDEX, cv2.CAP_AVFOUNDATION)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)

    # 露出安定
    for _ in range(6):
        cap.read()

    # 5秒間で最も明るいフレームを選ぶ
    start = time.time()
    best_frame, best_rgb, best_brightness = None, None, -1
    while time.time() - start < capture_time:
        ok, frame = cap.read()
        if not ok:
            continue
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        br = brightness(rgb)
        if br > best_brightness:
            best_frame, best_rgb, best_brightness = frame.copy(), rgb.copy(), br
    cap.release()

    if best_rgb is None:
        print("❌ カメラからフレームが取得できませんでした")
        return

    # 検出（Haar → HOG）
    boxes = haar_boxes(best_rgb)
    detector = "Haar"
    if not boxes:
        boxes = hog_boxes(best_rgb, up=1)
        detector = "HOGx1"

    # 出力準備
    vis = best_rgb.copy()
    stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_dir = Path("share_out")
    out_dir.mkdir(exist_ok=True)
    attachments = []
    report_lines = [
        f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
        f"Detections: {len(boxes)}",
        f"Detector: {detector}",
        ""
    ]

    # 認識＆描画＆切り出し
    names_for_tts = []
    if len(boxes):
        encs = face_recognition.face_encodings(best_rgb, known_face_locations=boxes)
        for idx, (box, enc) in enumerate(zip(boxes, encs), start=1):
            t, r, b, l = box
            cv2.rectangle(vis, (l, t), (r, b), (0, 255, 0), 2)
            label = "Unknown"
            meta = {}
            best = None

            if (known_encs is not None) and len(known_encs):
                d = face_recognition.face_distance(known_encs, enc)
                i = int(np.argmin(d))
                best = float(d[i])
                if best <= FACE_TOLERANCE:
                    label = owners[i]
                    meta = metas[i] if isinstance(metas[i], dict) else {}
                    vis = draw_label_jp(
                        vis, f"{label} ({best:.3f})", (l, max(5, t - 24))
                    )
                    y0 = b + 6
                    for j, (k, v) in enumerate(list(meta.items())[:3]):
                        vis = draw_label_jp(
                            vis, f"{k}: {v}", (l, y0 + j * 22)
                        )
                else:
                    vis = draw_label_jp(
                        vis, f"Unknown ({best:.3f})", (l, max(5, t - 24))
                    )
            else:
                vis = draw_label_jp(vis, "Unknown", (l, max(5, t - 24)))

            # 本文
            head = f"[Face {idx}] {label}" + (f"  dist={best:.3f}" if best is not None else "")
            report_lines.append(head)
            if meta:
                for k, v in list(meta.items())[:5]:
                    report_lines.append(f"  - {k}: {v}")
            report_lines.append("")

            # TTS用：名前抽出（Unknownは除外）
            if label != "Unknown":
                names_for_tts.append(label)

            # 顔切り出し保存→添付
            crop = best_frame[t:b, l:r]
            cp = out_dir / f"crop_{idx}_{stamp}.jpg"
            cv2.imwrite(str(cp), crop)
            attachments.append(cp)
    else:
        report_lines.append("[No face detected]")

    # オーバーレイ画像
    overlay_path = out_dir / f"snapshot_{stamp}.jpg"
    cv2.imwrite(str(overlay_path), cv2.cvtColor(vis, cv2.COLOR_RGB2BGR))
    attachments.insert(0, overlay_path)

    # === シンプルTTS（「〇〇を検出しました」だけ） ===
    try:
        if names_for_tts:
            tts_text = "、".join([f"{n}を検出しました" for n in names_for_tts])
        else:
            tts_text = "顔は検出されませんでした"
        voice_path = make_tts_audio(tts_text)
        attachments.append(voice_path)
        print("🎧 音声も添付しました:", voice_path)
    except Exception as e:
        print("⚠️ 音声生成に失敗:", e)

    # 送信（★ 件名を「〇〇を検出しました」形式に変更 ★）
    if names_for_tts:
        # 例: 「石岡和磨を検出しました」 or 「Aを検出しました、Bを検出しました」
        subject = "、".join([f"{n}を検出しました" for n in names_for_tts])
    else:
        subject = "顔は検出されませんでした"

    body = "\n".join(report_lines)
    send_via_mail_app(RECIPIENT, subject, body, attachments)

    print("✅ 自動送信完了:", RECIPIENT)
    print("件名:", subject)
    print("本文:\n" + body)
    print("添付:")
    for p in attachments:
        print(" -", p)


In [ ]:
identify_and_email(capture_time=3.0)